<a href="https://colab.research.google.com/github/caganze/SummerSchoolWorkshops/blob/main/Introduction_Classification_by_Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install astroquery
%%capture
!pip install astroquery

In [ ]:
from astroquery.sdss import SDSS  # enables direct queries to the SDSS database
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [ ]:
#plotting aesthetics
import matplotlib as mpl
import seaborn as sns
plt.style.use('fivethirtyeight')
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['figure.figsize'] = [8.0, 4.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 18
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'large'
mpl.rcParams['xtick.bottom']=True
mpl.rcParams['xtick.top']=True
mpl.rcParams['xtick.major.width']=0.9
mpl.rcParams['xtick.minor.width']=0.9
mpl.rcParams['ytick.major.width']=0.9
mpl.rcParams['ytick.minor.width']=0.9
mpl.rcParams['ytick.right']=True
mpl.rcParams['ytick.left']=True
mpl.rcParams['xtick.direction']='in'
mpl.rcParams['ytick.direction']='in'

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14) 
font = {'family' : 'Helvetica',
        'size'   : 14}
mpl.rc('font', **font)
mpl.rc('text', usetex=False)
mpl.rcParams['agg.path.chunksize'] = 10000


Let's select magnitud measurements for 100000 stars and galaxies from SDSS.


* p.mode = 1 select only the primary photometric detection of a source

* s.sciencePrimary = 1 select only the primary spectroscopic detection of a source (together with above, prevents duplicates)

* p.clean = 1 the SDSS clean flag excludes flagged observations and sources with non-detections

* s.class != 'QSO' removes potentially ambiguous QSOs from the training set




In [ ]:
#query a 10000 objects 
TSquery = """SELECT TOP 10000 
             p.psfMag_r, p.fiberMag_r, p.fiber2Mag_r, p.petroMag_r, 
             p.deVMag_r, p.expMag_r, p.modelMag_r, p.cModelMag_r, 
             s.class
             FROM PhotoObjAll AS p JOIN specObjAll s ON s.bestobjid = p.objid
             WHERE p.mode = 1 AND s.sciencePrimary = 1 AND p.clean = 1 AND s.class != 'QSO'
             ORDER BY p.objid ASC
               """
SDSSts = SDSS.query_sql(TSquery).to_pandas()
SDSSts.head()

In [ ]:
#quick visualization of the data 
fig, ax=plt.subplots(figsize=(6, 4))
sns.scatterplot(x="psfMag_r", y="modelMag_r",
            hue="class", palette=["#FF4136", "#0074D9"], style="class",
            data=SDSSts, ax=ax)

# Random Forest Classifier Example

Let's do some reformatting of the data to allow numerical spectral types \



In [ ]:
#some reformatting 
#let's use numerical labels
def numerical_class(c):
  if c==b'GALAXY':
    return 0
  elif c==b'STAR':
    return 1
  else:
    return -1
SDSSts['num_class']=SDSSts['class'].apply(numerical_class)

In [ ]:
SDSSts.head(5)

In [ ]:
len(SDSSts[SDSSts['num_class']==0]), len(SDSSts[SDSSts['num_class']==1])

Let's split the data into a a training, validation and test set 

This is a critical step in a setting up our machine learning model. 

Splitting the data into smaller these sets allows us to fit the model and evaluate how our machine learning model would perform on unseen/new data. ` sklearn ` has all these tools built in for us, we can just call them directly 

In [ ]:
from sklearn.model_selection import train_test_split
RSEED = 42  

#remove 
feats = list(SDSSts.columns)
feats.remove('class')
feats.remove('num_class')

X0 = np.array(SDSSts[feats]) #data
y0 = np.array(SDSSts['num_class']) #labels 

X_train, X_test, y_train, y_test = train_test_split(X0,y0,train_size=0.75, random_state=RSEED, shuffle=True)

Let's now build a random forest classifier model directly from Sklearn 

In [ ]:
#let's build a random forest classifier 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import precision_score

RF = RandomForestClassifier(n_estimators =50)
RF.fit(X_train, y_train) #fitting to the training set 
test_preds = RF.predict(X_test) #predict on the test 

print("The raw features produce a model with precision ~{:.4f} !".format(precision_score(y_test, test_preds)))

One can find out which magnitudes here are more important in distinguishing beteen stars and galaxies

In [ ]:
fig, ax=plt.subplots()

tree_feature_importances = (
    RF.feature_importances_)
    
sorted_idx = tree_feature_importances.argsort()

x_ticks = np.arange(0, len(feats))
ax.bar(x_ticks, tree_feature_importances[sorted_idx])
ax.set_xticklabels(np.array(feats)[sorted_idx], rotation='vertical')
ax.set_xticks(x_ticks)
ax.set_title("Random Forest Feature Importance ")

The most important magnitude alone cannot distinguish between stars and galaxies since their distributions (stars vs galaxies) overlap

In [ ]:
#let's look at the distribution of the most important feature
fig, ax=plt.subplots()
h=plt.hist(SDSSts.psfMag_r[SDSSts.num_class==0], bins='auto',\
           histtype='step', label='GALAXY', lw=3)
h=plt.hist(SDSSts.psfMag_r[SDSSts.num_class==1], bins='auto', \
           histtype='step', label='STAR', lw=3 )
ax.set(xlabel='PSF Mag r', ylabel='Number ')
ax.legend(loc='upper left')
#sure these to 

# Evaluating model & Optimizing Parameters: K-fold Cross- Validation and Grid Search


Let's use the idea of cross-validation to evaluate the performance of this  model on a new dataset 

In [ ]:
#manual k-fold cross validation
from numpy import array
from sklearn.model_selection import KFold

Let's perform a random search over the number of trees using sklearn functions

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Reformat our error function to be digestible by sklearn

In [ ]:
def perform_random_search(X_train, y_train):
  """
  This function perfoms a search over an set of models using cross-validation

  The best function is defined to have the highest precision
  """
  #default is 5-fold CV but can be modified
  param_dist = {'n_estimators': np.linspace(20, 500, 5, dtype=int)}            
  random_search = RandomizedSearchCV( RandomForestClassifier(), param_distributions=param_dist, n_jobs=-1, 
                                     verbose=True, random_state=RSEED, scoring='precision' )
  random_search.fit(X_train, y_train)
  print ('our best model has {} estimators'.format( random_search.best_estimator_.n_estimators))
  return random_search.best_estimator_


In [ ]:
#evaluate our best model
best_model=perform_random_search(X_train, y_train)

In [ ]:
best_model.fit(X_train, y_train)
test_preds = best_model.predict(X_test)

print("With an optimized mode, we get a precision of ~{:.4f} !".format(precision_score(y_test, test_preds)))

Challenge problem: replace  RandomizedSearchCV by GridSearchCV, compare results

> Indented block



In [ ]:
#insert code here 

# References & Sources

This notebook uses some material from the LSST-DSFP fellowship program Session 1 shorturl.at/tzMTY , sklearn documentation and astroquery documentation